<a href="https://colab.research.google.com/github/Otobi1/Bike-Demand-Prediction/blob/master/Bike_Demand_Prediction_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedShuffleSplit

import warnings
warnings.filterwarnings('ignore')

In [3]:
!wget http://archive.ics.uci.edu/ml/machine-learning-databases/00560/SeoulBikeData.csv

--2021-03-29 16:04:24--  http://archive.ics.uci.edu/ml/machine-learning-databases/00560/SeoulBikeData.csv
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 604166 (590K) [application/x-httpd-php]
Saving to: ‘SeoulBikeData.csv’

SeoulBikeData.csv   100%[===================>] 590.01K  --.-KB/s    in 0.07s   

2021-03-29 16:04:24 (8.68 MB/s) - ‘SeoulBikeData.csv’ saved [604166/604166]



In [5]:
df = pd.read_csv('./SeoulBikeData.csv', index_col='Date', encoding='unicode_escape')

data = df.copy()

In [8]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(data, data['Seasons']):
  strat_train_set = data.iloc[train_index]
  strat_test_set = data.iloc[test_index]

In [9]:
data = strat_train_set.drop('Rented Bike Count', axis=1)
data_labels = strat_train_set['Rented Bike Count'].copy()
data

,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(°C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day
Date,,,,,,,,,,,,
29/06/2018,9,22.9,86,1.7,538,20.4,0.76,0.0,0.0,Summer,No Holiday,Yes
13/01/2018,18,-2.6,73,1.0,684,-6.7,0.01,0.0,0.0,Winter,No Holiday,Yes
26/11/2018,22,6.2,70,0.4,474,1.1,0.00,0.0,0.0,Autumn,No Holiday,Yes
05/06/2018,8,21.5,58,1.1,1021,12.8,1.21,0.0,0.0,Summer,No Holiday,Yes
01/06/2018,21,23.5,44,1.6,2000,10.5,0.00,0.0,0.0,Summer,No Holiday,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...
30/11/2018,3,-0.1,69,0.9,1338,-5.1,0.00,0.0,0.0,Autumn,No Holiday,Yes
21/06/2018,13,27.3,33,3.4,1921,9.6,3.42,0.0,0.0,Summer,No Holiday,Yes
22/02/2018,4,-2.4,58,0.3,2000,-9.5,0.00,0.0,0.0,Winter,No Holiday,Yes


In [11]:
data_cat = data[['Seasons', 'Holiday', 'Functioning Day']]
data_cat.head()

,Seasons,Holiday,Functioning Day
Date,,,
29/06/2018,Summer,No Holiday,Yes
13/01/2018,Winter,No Holiday,Yes
26/11/2018,Autumn,No Holiday,Yes
05/06/2018,Summer,No Holiday,Yes
01/06/2018,Summer,No Holiday,Yes


In [12]:
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder()
data_cat_1hot = cat_encoder.fit_transform(data_cat)
data_cat_1hot

<7008x8 sparse matrix of type '<class 'numpy.float64'>'
	with 21024 stored elements in Compressed Sparse Row format>

In [13]:
data_cat_1hot.toarray()[:5]

array([[0., 0., 1., 0., 0., 1., 0., 1.],
       [0., 0., 0., 1., 0., 1., 0., 1.],
       [1., 0., 0., 0., 0., 1., 0., 1.],
       [0., 0., 1., 0., 0., 1., 0., 1.],
       [0., 0., 1., 0., 0., 1., 0., 1.]])

In [14]:
cat_encoder.categories_

[array(['Autumn', 'Spring', 'Summer', 'Winter'], dtype=object),
 array(['Holiday', 'No Holiday'], dtype=object),
 array(['No', 'Yes'], dtype=object)]

In [15]:
num_data = data.iloc[:, :-3]
num_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7008 entries, 29/06/2018 to 08/10/2018
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Hour                       7008 non-null   int64  
 1   Temperature(°C)            7008 non-null   float64
 2   Humidity(%)                7008 non-null   int64  
 3   Wind speed (m/s)           7008 non-null   float64
 4   Visibility (10m)           7008 non-null   int64  
 5   Dew point temperature(°C)  7008 non-null   float64
 6   Solar Radiation (MJ/m2)    7008 non-null   float64
 7   Rainfall(mm)               7008 non-null   float64
 8   Snowfall (cm)              7008 non-null   float64
dtypes: float64(6), int64(3)
memory usage: 707.5+ KB


In [16]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy = 'median')
imputer.fit(num_data)

SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='median', verbose=0)

In [17]:
imputer.statistics_

array([1.200e+01, 1.380e+01, 5.700e+01, 1.500e+00, 1.698e+03, 5.200e+00,
       1.000e-02, 0.000e+00, 0.000e+00])

In [18]:
data.median().values

array([1.200e+01, 1.380e+01, 5.700e+01, 1.500e+00, 1.698e+03, 5.200e+00,
       1.000e-02, 0.000e+00, 0.000e+00])

In [20]:
X = imputer.transform(num_data)
X

array([[ 9.00e+00,  2.29e+01,  8.60e+01, ...,  7.60e-01,  0.00e+00,
         0.00e+00],
       [ 1.80e+01, -2.60e+00,  7.30e+01, ...,  1.00e-02,  0.00e+00,
         0.00e+00],
       [ 2.20e+01,  6.20e+00,  7.00e+01, ...,  0.00e+00,  0.00e+00,
         0.00e+00],
       ...,
       [ 4.00e+00, -2.40e+00,  5.80e+01, ...,  0.00e+00,  0.00e+00,
         0.00e+00],
       [ 2.00e+01,  3.40e+00,  3.70e+01, ...,  0.00e+00,  0.00e+00,
         0.00e+00],
       [ 4.00e+00,  1.04e+01,  6.10e+01, ...,  0.00e+00,  0.00e+00,
         0.00e+00]])

In [21]:
data_tr = pd.DataFrame(X, columns=num_data.columns, index=num_data.index)

data_tr.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7008 entries, 29/06/2018 to 08/10/2018
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Hour                       7008 non-null   float64
 1   Temperature(°C)            7008 non-null   float64
 2   Humidity(%)                7008 non-null   float64
 3   Wind speed (m/s)           7008 non-null   float64
 4   Visibility (10m)           7008 non-null   float64
 5   Dew point temperature(°C)  7008 non-null   float64
 6   Solar Radiation (MJ/m2)    7008 non-null   float64
 7   Rainfall(mm)               7008 non-null   float64
 8   Snowfall (cm)              7008 non-null   float64
dtypes: float64(9)
memory usage: 707.5+ KB


In [22]:
num_data.head()

,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(°C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm)
Date,,,,,,,,,
29/06/2018,9,22.9,86,1.7,538,20.4,0.76,0.0,0.0
13/01/2018,18,-2.6,73,1.0,684,-6.7,0.01,0.0,0.0
26/11/2018,22,6.2,70,0.4,474,1.1,0.00,0.0,0.0
05/06/2018,8,21.5,58,1.1,1021,12.8,1.21,0.0,0.0
01/06/2018,21,23.5,44,1.6,2000,10.5,0.00,0.0,0.0


In [23]:
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import StandardScaler

numerics = ['float64', 'int64']

num_data = data_tr.select_dtypes(include=numerics)

num_pipeline = Pipeline([
                         ('Imputer', SimpleImputer(strategy='median')), 
                         ('std_scaler', StandardScaler())
])

num_data_tr = num_pipeline.fit_transform(num_data)
num_data_tr[0]

array([-0.35968275,  0.83664277,  1.36677375, -0.02323267, -1.4890478 ,
        1.24640775,  0.22153702, -0.13145357, -0.1724833 ])

In [24]:
from sklearn.compose import ColumnTransformer

num_attrs = list(num_data)
cat_attrs = ['Seasons', 'Holiday', 'Functioning Day']

full_pipeline = ColumnTransformer([
                                   ('num', num_pipeline, num_attrs), 
                                   ('cat', OneHotEncoder(), cat_attrs), 
])

prepared_data = full_pipeline.fit_transform(data)
prepared_data

array([[-0.35968275,  0.83664277,  1.36677375, ...,  1.        ,
         0.        ,  1.        ],
       [ 0.93947602, -1.29688457,  0.72763035, ...,  1.        ,
         0.        ,  1.        ],
       [ 1.51687991, -0.56060847,  0.58013572, ...,  1.        ,
         0.        ,  1.        ],
       ...,
       [-1.08143762, -1.28015103, -0.0098428 , ...,  1.        ,
         0.        ,  1.        ],
       [ 1.22817796, -0.79487814, -1.04230521, ...,  1.        ,
         0.        ,  1.        ],
       [-1.08143762, -0.20920397,  0.13765183, ...,  1.        ,
         0.        ,  1.        ]])